In [25]:
import torch
import torch.nn as nn


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input channel, 6 output channels, 5x5 kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 6 input channels, 6 output channels, 5x5 kernel
        self.pool1 = nn.MaxPool2d(2, 2)

        # 6 input channels, 16 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 16 input channels, 16 output channels, 5x5 kernel
        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)  # input shape: (batch_size, 1, 28, 28)
        x = torch.relu(x)  # output shape: (batch_size, 6, 24, 24)
        x = self.pool1(x)  # output shape: (batch_size, 6, 12, 12)

        x = self.conv2(x)  # input shape: (batch_size, 6, 12, 12)
        x = torch.relu(x)  # output shape: (batch_size, 16, 8, 8)
        x = self.pool2(x)  # output shape: (batch_size, 16, 4, 4)

        x = x.view(x.shape[0], -1)  # output shape: (batch_size, 16*4*4)
        x = self.fc1(x)  # output shape: (batch_size, 120)
        x = torch.relu(x)
        x = self.fc2(x)  # output shape: (batch_size, 84)
        x = torch.relu(x)
        x = self.fc3(x)  # output shape: (batch_size, 10)

        return x


def print_params(model):
    count = 0
    for name, param in model.named_parameters():
        count += param.numel()
        print(name, param.shape)
    print('total parameters:', count)


def print_forward(model, x):
    print('input x to model:', x.shape)
    for name, layer in model.named_children():
        if name == 'fc1':
            x = x.view(x.shape[0], -1)
        x = layer(x)
        print(f'{name} after size: {x.shape}')


model = LeNet()
print(model)
print('---')
print_params(model)

x = torch.randn(64, 1, 28, 28)
print_forward(model, x)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
---
conv1.weight torch.Size([6, 1, 5, 5])
conv1.bias torch.Size([6])
conv2.weight torch.Size([16, 6, 5, 5])
conv2.bias torch.Size([16])
fc1.weight torch.Size([120, 256])
fc1.bias torch.Size([120])
fc2.weight torch.Size([84, 120])
fc2.bias torch.Size([84])
fc3.weight torch.Size([10, 84])
fc3.bias torch.Size([10])
total parameters: 44426
input x to model: torch.Size([64, 1, 28, 28])
conv1 after size: torch.Size([64, 6, 24, 24])
pool1 after size: torch.Size([64, 6, 12, 12])
conv2 after size: torch.Size([64, 16, 8, 